In [4]:
import pandas as pd
import sqlite3
from flask import Flask, render_template, jsonify
import folium

app = Flask(__name__)

# Database connection function
def get_db_connection():
    conn = sqlite3.connect('database.db')
    conn.row_factory = sqlite3.Row
    return conn

# Create database and tables
def init_db():
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.executescript('''
    CREATE TABLE IF NOT EXISTS people (
        "Income Group"                             VARCHAR(6)      NOT NULL,
        "Per Capita GDP"                           DECIMAL(10,2)   NOT NULL,
        "Population (millions)"                    DECIMAL(10,2)   NOT NULL,
        "Human Development Index (HDI)"            DECIMAL(10,2)   NOT NULL,
        "Life Expectancy"                          DECIMAL(10,2)   NOT NULL
    );
    
    CREATE TABLE IF NOT EXISTS land (
        "Country"                                  VARCHAR(50)     NOT NULL,
        "Region"                                   VARCHAR(50)     NOT NULL,
        "SDGi"                                     DECIMAL(10,2)   NOT NULL,
        "Cropland Footprint"                       DECIMAL(10,2)   NOT NULL,
        "Grazing Footprint"                        DECIMAL(10,2)   NOT NULL,
        "Forest Product Footprint"                 DECIMAL(10,2)   NOT NULL,
        "Carbon Footprint"                         DECIMAL(10,2)   NOT NULL,
        "Fish Footprint"                           DECIMAL(10,2)   NOT NULL,
        "Built up land"                            DECIMAL(10,2)   NOT NULL,
        "Total Ecological Footprint (Consumption)" DECIMAL(10,6)   NOT NULL,
        "Cropland"                                 DECIMAL(10,2)   NOT NULL,
        "Grazing land"                             DECIMAL(10,2)   NOT NULL,
        "Forest land"                              DECIMAL(10,2)   NOT NULL,
        "Fishing ground"                           DECIMAL(10,2)   NOT NULL,
        "Total biocapacity"                        DECIMAL(10,2)   NOT NULL,
        "Ecological (Deficit) or Reserve"          DECIMAL(10,2)   NOT NULL,
        "Number of Earths required"                DECIMAL(10,2)   NOT NULL,
        "Number of Countries required"             DECIMAL(10,2)   NOT NULL
    );
    ''')
    conn.commit()
    conn.close()

# Initialize the database
init_db()

@app.route('/')
def index():
    # Load data from the database
    conn = get_db_connection()
    people = conn.execute('SELECT * FROM people').fetchall()
    land = conn.execute('SELECT * FROM land').fetchall()
    conn.close()

    # Create a DataFrame
    df_people = pd.DataFrame(people)
    df_land = pd.DataFrame(land)

    return render_template('index.html', people=df_people, land=df_land)

@app.route('/map')
def show_map():
    # Example data for map visualization (replace with actual data)
    data = {
        'Country': ['United States', 'China', 'India'],
        'Ecological Footprint': [8.5, 2.1, 1.2],
        'Latitude': [37.0902, 35.8617, 20.5937],
        'Longitude': [-95.7129, 104.1954, 78.9629]
    }
    df = pd.DataFrame(data)

    # Create a base map
    m = folium.Map(location=[20, 0], zoom_start=2)

    # Add data to the map
    for i, row in df.iterrows():
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=row['Ecological Footprint'],
            popup=row['Country'],
            color='blue',
            fill=True,
            fill_color='blue'
        ).add_to(m)

    # Save the map to an HTML file and read it
    m.save('templates/map.html')
    with open('templates/map.html', 'r') as f:
        map_html = f.read()

    return map_html

@app.route('/data')
def get_data():
    conn = get_db_connection()
    people = conn.execute('SELECT * FROM people').fetchall()
    land = conn.execute('SELECT * FROM land').fetchall()
    conn.close()

    # Convert data to dictionaries
    people_data = [dict(row) for row in people]
    land_data = [dict(row) for row in land]

    return jsonify({'people': people_data, 'land': land_data})

if __name__ == '__main__':
    app.run(debug=False)  # Set debug=False to avoid using watchdog


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
